In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)  
import numpy as np 
import matplotlib.pyplot as plt
from PIL import Image, ImageFile, UnidentifiedImageError
import os
import pycaret 
ImageFile.LOAD_TRUNCATED_IMAGES = True 
import torch 
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torchvision 
from torchvision import datasets, models, transforms 
from tempfile import TemporaryDirectory 
from torch.utils.data import DataLoader 
from torch.utils.data import Dataset
from torchvision.models import resnet18, resnet50, VGG19_Weights
import warnings
warnings.filterwarnings("ignore") 
# from sklearn.model_selection import train_test_split
# import mlflow                                   
# import mlflow.pytorch
# Ignorer tous les avertissements
# from torchmetrics.classification import Accuracy
# from efficientnet_pytorch import EfficientNet
# import pretrainedmodels
# import seaborn as sns
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

### CONCATENATION DES FICHIERS 

In [4]:
df = pd.read_csv('/opt/spark-apps/programs/Gohou/Data/Tabes/Final.csv')
df = df.drop(columns=['Unnamed: 0'], axis =1) 
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df.shape 

(6497, 1085)

### ANALYSE EN COMPOSANTE PRINCIPALE

### CHARGEMENT DES MODULES NECESSAIRE AU MACHINE LEARNING 

In [5]:
import time
import xgboost as xgb
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, f1_score
from sklearn.model_selection import cross_val_score, KFold,cross_val_predict, train_test_split,GridSearchCV
from sklearn.linear_model import LinearRegression,LogisticRegression,ElasticNet, Ridge, Lasso
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

### REECHANTILLONNAGE DES DONNEES 

In [6]:
from imblearn.over_sampling import SMOTE
from collections import Counter
X = df.drop(columns='cluster')  # Variables indépendantes
y = df['cluster'] 

# # Séparation des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42,stratify=y)

# Application de SMOTE pour suréchantillonner la classe minoritaire
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Vérifier la distribution des classes après suréchantillonnage
print("Distribution des classes avant suréchantillonnage :", Counter(y_train))
print("Distribution des classes après suréchantillonnage :", Counter(y_train_resampled))

Distribution des classes avant suréchantillonnage : Counter({0: 2385, 1: 1239, 2: 584, 3: 339})
Distribution des classes après suréchantillonnage : Counter({0: 2385, 2: 2385, 1: 2385, 3: 2385})


### CHOIX DES MEILLEURS HYPERPARAMETRES PAR MODELES 

In [9]:
a = int(np.sqrt(len(X_train_resampled.columns))) 
dico = {
    'logistic': LogisticRegression(),
    'elastic': LogisticRegression(penalty="elasticnet", solver="saga"),  #met beaucoup de temps 
    'ridge': LogisticRegression(penalty="l2"),
    'lasso': LogisticRegression(penalty="l1"),
    'Arbre': DecisionTreeClassifier(),
    'Random': RandomForestClassifier(), 
    'xgb': xgb.XGBClassifier(),  #surapprend 
    'knn': KNeighborsClassifier(),
    'Gradient_boosting': GradientBoostingClassifier(),
    'adaboost_classifier': AdaBoostClassifier(),
    'Bagging': BaggingClassifier(DecisionTreeClassifier(max_depth=7)),  #surapprend
    'SVM': SVC(),
    'LDA': LinearDiscriminantAnalysis(),
    'QDA': QuadraticDiscriminantAnalysis(), #surapprend (meilleur => accuracy 0.52)
    'lightgbm': LGBMClassifier(),  # Ajout de LightGBM
    'catboost': CatBoostClassifier(verbose=0)  # Ajout de CatBoost
}

param = [ 
    {"C": [0.01], "max_iter": [10000]},
    {"C": [0.01], "l1_ratio": [0.1, 0.5, 0.7], "max_iter": [10000]},
    {"C": [0.01], "solver": ["sag", "lbfgs", "liblinear"], "max_iter": [10000]}, # C grand surapprentissage 
    {"C": [0.01], "solver": ["liblinear"], "max_iter": [10000]},
    {"max_depth": [5, 10], "min_samples_split": [10, 15], "min_samples_leaf": [5, 7]},
    {"n_estimators": [100, 200], "min_samples_leaf": [5, 7], "max_features": [0.6, 0.5], "max_depth": [5, 7]},

# {'max_depth': 10,
#  'max_features': 0.8,
#  'min_samples_leaf': 3,
#  'n_estimators': 300}
    
    {'colsample_bytree': [0.5, 0.7], "n_estimators": [50, 100], "max_depth": [2, 3], "objective": ["multi:softmax"], "num_class": [4], "subsample": [0.5, 0.6]}, 
    {'n_neighbors': [5,7], "algorithm": ["auto","ball_tree"]}, # k petit surapprentissage 
    {"n_estimators": [800, 1000], "max_depth": [3], "learning_rate": [0.01, 0.05], "min_samples_split": [10, 15], "min_samples_leaf": [5, 7], "subsample": [0.5, 0.6]},
    {"n_estimators": [800, 1000], "learning_rate": [0.05, 0.08], "algorithm": ["SAMME", "SAMME.R"]},
    {"n_estimators": [50], "max_features": [0.5, 0.6], "max_samples":[0.5, 0.6]},
    {"kernel": ['linear', 'rbf'], "C": [0.1], "gamma": ["scale", "auto"]}, #C grand => surapprentisage
    {"solver": ["eigen", "lsqr"], "shrinkage": ["auto"]}, 
    {"reg_param": [0.7, 0.9], "tol": [1e-2, 0.1]},  
    {"n_estimators": [500, 700], "learning_rate": [0.01, 0.03], "max_depth": [2, 3], "num_leaves": [10, 15], "colsample_bytree": [0.5, 0.6], "force_row_wise": [True], 'verbose': [-1]},
    
# {'colsample_bytree': 0.5,
#  'force_row_wise': True,
#  'learning_rate': 0.05,
#  'max_depth': 3,
#  'n_estimators': 1000,
#  'num_leaves': 20,
#  'verbose': -1}
    
    {"iterations": [1000, 800], "depth": [3], "learning_rate": [0.01, 0.05], "l2_leaf_reg": [7, 10]}
] 

In [10]:
def train_model(nom_model, dico, param, X_train_resampled, y_train_resampled, kf, out):
    model = dico[nom_model]
    model_cv = GridSearchCV(model, param_grid=param, cv=kf, scoring="f1_weighted", error_score="raise")  # Utiliser "f1" au lieu de "f1_score"
    temps_debut = time.time() 
    model_cv.fit(X_train_resampled, y_train_resampled) 
    temps_fin = time.time()
    # temps_execution = temps_fin - temps_debut
    out[nom_model]={"best_model":model_cv.best_estimator_,"meilleurs_parametres":model_cv.best_params_, "f1_score": model_cv.best_score_,"temps":[time.ctime(temps_debut), time.ctime(temps_fin)]}
    print(f'-----------------  Entrainement de {nom_model} est terminé  -----------------')

In [11]:
import threading
kf = KFold(n_splits=5, shuffle=True, random_state=42)
out = {}
threads = []

# Launch a thread for each model in the dictionary
for i, nom_model in enumerate(dico):
    param_v = param[i]
    thread = threading.Thread(target=train_model, args=(nom_model, dico, param_v, X_train_resampled, y_train_resampled, kf, out))
    threads.append(thread)
    thread.start() 

-----------------  Entrainement de lasso est terminé  -----------------
-----------------  Entrainement de knn est terminé  -----------------
-----------------  Entrainement de Arbre est terminé  -----------------
-----------------  Entrainement de elastic est terminé  -----------------
-----------------  Entrainement de LDA est terminé  -----------------
-----------------  Entrainement de logistic est terminé  -----------------
-----------------  Entrainement de SVM est terminé  -----------------
-----------------  Entrainement de ridge est terminé  -----------------
-----------------  Entrainement de Bagging est terminé  -----------------
-----------------  Entrainement de xgb est terminé  -----------------
-----------------  Entrainement de catboost est terminé  -----------------
-----------------  Entrainement de QDA est terminé  -----------------
-----------------  Entrainement de adaboost_classifier est terminé  -----------------
-----------------  Entrainement de lightgbm est te

In [141]:
out_1 = pd.DataFrame.from_dict(out, orient='index')
out_1

,best_model,meilleurs_parametres,f1_score,temps
lasso,"LogisticRegression(C=0.01, max_iter=10000, pen...","{'C': 0.01, 'max_iter': 10000, 'solver': 'libl...",0.579699,"[Thu Oct 24 18:10:18 2024, Thu Oct 24 18:18:10..."
knn,KNeighborsClassifier(),"{'algorithm': 'auto', 'n_neighbors': 5}",0.560430,"[Thu Oct 24 18:10:19 2024, Thu Oct 24 19:36:22..."
Arbre,"DecisionTreeClassifier(max_depth=10, min_sampl...","{'max_depth': 10, 'min_samples_leaf': 5, 'min_...",0.514658,"[Thu Oct 24 18:10:19 2024, Thu Oct 24 20:02:42..."
elastic,"LogisticRegression(C=0.01, l1_ratio=0.1, max_i...","{'C': 0.01, 'l1_ratio': 0.1, 'max_iter': 10000}",0.715944,"[Thu Oct 24 18:10:18 2024, Thu Oct 24 21:42:36..."
LDA,"LinearDiscriminantAnalysis(shrinkage='auto', s...","{'shrinkage': 'auto', 'solver': 'eigen'}",0.680674,"[Thu Oct 24 18:10:19 2024, Thu Oct 24 22:00:04..."
logistic,"LogisticRegression(C=0.01, max_iter=10000)","{'C': 0.01, 'max_iter': 10000}",0.721789,"[Thu Oct 24 18:10:18 2024, Fri Oct 25 01:41:42..."
SVM,"SVC(C=0.1, kernel='linear')","{'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}",0.791728,"[Thu Oct 24 18:10:19 2024, Fri Oct 25 01:43:43..."
ridge,"LogisticRegression(C=0.01, max_iter=10000)","{'C': 0.01, 'max_iter': 10000, 'solver': 'lbfgs'}",0.721789,"[Thu Oct 24 18:10:18 2024, Fri Oct 25 02:22:56..."
Bagging,"(DecisionTreeClassifier(max_depth=7, random_st...","{'max_features': 0.5, 'max_samples': 0.6, 'n_e...",0.699893,"[Thu Oct 24 18:10:19 2024, Fri Oct 25 02:35:12..."
xgb,"XGBClassifier(base_score=None, booster=None, c...","{'colsample_bytree': 0.7, 'max_depth': 3, 'n_e...",0.767437,"[Thu Oct 24 18:10:19 2024, Fri Oct 25 04:09:37..."


In [47]:
out_1.loc["QDA","meilleurs_parametres"] 

{'reg_param': 0.7, 'tol': 0.01}

In [ ]:
# Pour s'assurer que tous les threads individuelles soient terminés avant que le thread principal prenne le relai. 
# On en pas besoin si on sait à quel moment tous nos threads vont tous se stopper 

for thread in threads:
    thread.join() 
print(out) 

### CHOIX DU MEILLEUR MODELE 

In [108]:
def train_best(out, result, position): 
    kf = KFold(n_splits=10, shuffle=True, random_state=42) 
    model = list(out.values())[position]["best_model"] 
    nom = list(out.keys())[position]  
    predictions = cross_val_predict(model, X_train_resampled, y_train_resampled, cv = kf)
    result[nom] = {"Modele":model, "F1_Score": f1_score(y_train_resampled, predictions, average='weighted')} 
    print(f'----------------- Entrainement de {nom} est terminé --------------------') 

#### **On fait Ici des entrainement des modèles de façon paralleles en utilisant des threads**

In [110]:
result = {}
threads_best = [] 

# Launch a thread for each model in the dictionary
for i, nom_model in enumerate(out):
    position = i
    thread1 = threading.Thread(target=train_best, args=(out, result, position))
    threads_best.append(thread1)
    thread1.start()   

----------------- Entrainement de knn est terminé --------------------
----------------- Entrainement de Arbre est terminé --------------------
----------------- Entrainement de lasso est terminé --------------------
----------------- Entrainement de LDA est terminé --------------------
----------------- Entrainement de Bagging est terminé --------------------
----------------- Entrainement de Random est terminé --------------------
----------------- Entrainement de catboost est terminé --------------------
----------------- Entrainement de xgb est terminé --------------------
----------------- Entrainement de lightgbm est terminé --------------------
----------------- Entrainement de adaboost_classifier est terminé --------------------


In [21]:
df_result = pd.DataFrame.from_dict(result, orient='index') 
df_result 

NameError: name 'result' is not defined

In [146]:
import pickle

# Enregistrer le modèle
with open('/opt/spark-apps/programs/Gohou/Data/Tabes/New_trans/Random.pkl', 'wb') as file:
    pickle.dump(out_1.loc['Random']["best_model"], file)  

In [177]:
# result['catboost']["Modele"].save_model('/opt/spark-apps/programs/Gohou/Data/Tabes/catboost.cbm')
cat = CatBoostClassifier()
cat.load_model('/opt/spark-apps/programs/Gohou/Data/Tabes/catboost.cbm') 

In [178]:
cat.predict(X_train_resampled)
# f1_test = f1_score(y_test, pred, average='weighted')
# X_train_resampled, y_train_resampled
f1_test = f1_score(y_train_resampled, pred, average='weighted')
print(f1_test) 

1.0


In [124]:
# Charger le modèle plus tard
with open('/opt/spark-apps/programs/Gohou/Data/Tabes/New_trans/Random.pkl', 'rb') as file:
    loaded_model = pickle.load(file) 

In [120]:
a = X_test.reset_index(drop=True)
x = a.loc[0,:]

PC1    -3.870732
PC2     8.968597
PC3    -8.871503
PC4    -9.112597
PC5   -10.086656
Name: 0, dtype: float64

In [164]:
# X_test.head()

In [130]:
pr = loaded_model.predict(a.loc[7,:].values.reshape(1, -1))
print(pr)

[2]


In [ ]:
# Join all threads (wait for them to finish)
for thread1 in threads_best:
    thread1.join()
print(result) 

### AFFICHER LE MEILLEUR DES MEILLEURS PUIS ENTRAINEMENT SUR TOUTE LA DONNEES

In [89]:
meilleur_modele = df_result.loc[df_result["F1_Score"].idxmax()] 
meilleur_modele

Modele      LogisticRegression(C=1, max_iter=9000, penalty...
F1_Score                                             0.712651
Name: lasso, dtype: object

### Tester Le meilleur modele 

In [145]:
pred = out_1.loc["Random","best_model"].predict(X_test)
f1_test = f1_score(y_test, pred, average='weighted')
# X_train_resampled, y_train_resampled
# f1_test = f1_score(y_train_resampled, pred, average='weighted')
# f1_test = accuracy_score(y_test, pred)
print(f1_test)  

0.4564108862131048


In [96]:
a = pd.DataFrame({'True_label': y_test}, index=X_test.index)
b = pd.DataFrame({'Predict_label': list(pred)}, index=X_test.index)
c = pd.concat([X_test, a, b], axis=1)

In [136]:
df = c.copy() 
df = df.reset_index(drop=True)
df['Borne_inf'] = 500
df['Borne_sup'] = 333903
df['Moyenne'] = (500 + 333903)/2
for i in range(len(df)):
    if df.loc[i,'Predict_label'] == 1:
        df.loc[i, 'Borne_inf'] = 334000  
        df.loc[i, 'Moyenne'] = (334000 + 725650)/2
        df.loc[i, 'Borne_sup'] = 725650
    if df.loc[i,'Predict_label'] == 2:
        df.loc[i, 'Borne_inf'] = 726000
        df.loc[i, 'Moyenne'] = (726000 + 1236650)/2
        df.loc[i, 'Borne_sup'] = 1236650
    if df.loc[i,'Predict_label'] == 3:
        df.loc[i, 'Borne_inf'] = 1236650
        df.loc[i, 'Moyenne'] = (1236650 + 1883000)/2
        df.loc[i, 'Borne_sup'] = 1883000 

new_row = {col: '' for col in df.columns.difference(['Borne_inf', 'Borne_sup', 'Moyenne'])}
new_row['Borne_inf'] = df['Borne_inf'].mean()
new_row['Borne_sup'] = df['Borne_sup'].mean()
new_row['Moyenne'] = df['Moyenne'].mean()

# # Ajouter la nouvelle ligne au DataFrame
df.loc['Total'] = new_row 

In [140]:
df.tail(10)

PC1        PC2       PC3        PC4        PC5       PC6  \
1941    3.665545   9.095828 -4.009795 -13.879499 -10.923841  1.708179   
1942  -15.839944   7.585426  -2.99669   2.297182   0.193464 -4.716517   
1943    5.700526  -0.883723  0.766071   6.908685  -1.196897  1.515115   
1944    0.520782   0.023957 -7.124503   6.149444  -8.254216 -0.361884   
1945   -5.926856   0.688157 -7.093714   3.022891  10.765711 -3.850114   
1946   -6.304599 -18.387569 -2.105498  -5.686735   2.930576 -4.797416   
1947    7.487914 -16.659096  5.914499    1.81671   3.713782 -7.380473   
1948   -6.013292   3.402249 -5.238185  -4.783087  -1.121614 -1.438399   
1949   11.904688   -1.54971 -5.058869   1.758273   6.446852 -4.879979   
Total                                                                   

             PC7       PC8       PC9      PC10      PC11      PC12      PC13  \
1941   -6.440113  2.282255 -3.236435  -0.20193  -0.08404  1.716248 -0.821689   
1942    8.104228  2.603289  5.209993 -1.014898 -3.687687  1.687171 -0.153111   
1943  -14.981338 -2.621635  0.277191 -2.557242 -0.839728  5.285498 -3.165834   
1944    7.887145  0.388438 -2.295727 -0.008465  5.761222 -1.628289  -1.37574   
1945   -4.074808 -0.718308  5.181962  2.039703  8.350163   5.68959 -2.605559   
1946   -7.286451 -1.177212 -5.009652  -3.64464  9.391969  2.002526  1.255135   
1947    2.218108 -6.966858 -5.937964  5.434481  -0.95742  1.218868 -4.746692   
1948    13.24069   8.32433  3.759062  1.995231 -4.033427  4.954855  3.384132   
1949  -10.180599 -0.066413 -6.541351  2.601849  9.220267  2.401396   1.04566   
Total                                                                          

           PC14      PC15      PC16      PC17      PC18      PC19      PC20  \
1941   4.202562 -1.566287  1.526717 -1.118187 -3.338662  8.141715 -4.928181   
1942   -1.78461 -5.635216  2.013104  0.502348  0.539406  1.820933  1.359291   
1943  -3.330861  1.492915  1.188072  3.862789  4.021513 -3.311536  2.802777   
1944  -2.439228  0.696374 -0.119979  0.804483  3.144221 -3.325675  7.843043   
1945   4.259637  1.781938   2.11256  0.261466 -0.535581 -0.836843  0.804098   
1946   4.019662 -4.722135  0.778979  1.120782  2.639891  0.949089 -1.151704   
1947  -1.227955  3.391083  6.588222 -5.469644 -5.941451 -0.852401 -1.988315   
1948   3.681146  1.548071 -2.609439 -0.915581 -2.188457 -0.615164   1.54478   
1949   0.073814 -5.003687  0.773507 -1.688102  2.144756 -1.967239 -4.637638   
Total                                                                         

           PC21      PC22      PC23      PC24      PC25      PC26      PC27  \
1941   1.268262 -0.100405 -7.156666  0.920758  3.863543  3.091602  2.457195   
1942   5.552146 -2.447609 -0.114713  1.027984 -2.789542  2.212824 -2.428403   
1943   3.759271 -6.043781 -4.162469 -6.718621 -7.876275 -0.919048  2.769018   
1944   6.685664    3.0526 -1.578796 -1.946466  0.847929   0.56739  0.698753   
1945   4.923455    1.4203  1.328689 -4.881835  0.492426  2.348593  -0.98121   
1946   1.131152 -2.657743 -1.875129  3.423873  0.773759 -0.524842 -0.873795   
1947   1.263211  0.675473  -3.07064  1.336336 -2.423624   1.87795 -1.480181   
1948  -1.397478  1.456877 -6.529143 -0.754757  9.801327 -4.946612  3.037695   
1949   1.643582  0.862344  4.470598 -0.679466  2.432128  5.787196  3.506613   
Total                                                                         

           PC28      PC29      PC30      PC31      PC32      PC33      PC34  \
1941   2.644689 -3.340812  0.455845   3.33621  0.509698 -3.699761 -3.202855   
1942   4.159841 -4.138441 -2.641676 -4.753854  0.222024 -1.551672 -3.157388   
1943   1.614114 -0.542611  -4.80053  1.787622  1.880452 -0.661166  -0.95703   
1944   0.551148  -3.12919  2.582674  2.114465 -6.750435   1.72054  0.464496   
1945   3.208203  2.128548  4.917801 -5.035291  0.187607  1.061922 -2.053172   
1946  -1.712941  0.089051 -2.559842   0.94227  -0.55768 -0.539742 -3.846752   
1947  -2.696823 -1.224864  0.627753

### Courbe d'apprentissage 

In [106]:
# param1 = {"n_estimators": [100, 80], "max_features": [0.7, 0.8], "max_samples":[0.7, 0.8]}
# kf1 = KFold(n_splits= 4, shuffle=True, random_state=42) 
# mo = GridSearchCV( BaggingClassifier(DecisionTreeClassifier()), param_grid=param1, cv=kf1, scoring="f1_weighted", error_score="raise")
# mo.fit(X_train_resampled, y_train_resampled)

# predictions = cross_val_predict(mo, X_train_resampled, y_train_resampled, cv = kf1) 
# f1_train = f1_score(y_train_resampled, predictions, average='weighted')
# print(f"F1 Score d'entraînement : {f1_train}")

# pred_test = mo.best_estimator_.predict(X_test)
# f1_test = f1_score(y_test, pred_test, average='weighted')
# print(f"F1 Score de test : {f1_test}")

# print(f"Meilleurs paramètres : {mo.best_params_}") 